*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*


# Number of Transactions per Year

In [9]:
from utils import increment_dictionary, sort_dictionary_by_values, path_csv, get_data, get_year
from csv_utils import make_csv
import pandas as pd 

def num_of_trans_per_year():
    _, rows = get_data()
    d={}
    
    for _, transaction in rows:
        d = increment_dictionary(d, get_year(transaction['transaction_date']))

    filename = "num_of_trans_per_year"
    key_header = "year"
    value_header = "number_of_transactions"

    d = sort_dictionary_by_values(d)
    
    wd = make_csv(path_csv, filename, d, [key_header, value_header])
    df = pd.read_csv(wd)
    print(df.head(5))

    return d 

num_of_trans_per_year_res = num_of_trans_per_year()

   year  number_of_transactions
0  2020                    1533
1  2018                    1395
2  2017                    1372
3  2015                    1152
4  2019                    1037


# Number of Transactions per Person 
_Not controlling for number of years in position or size of transaction._ For each official, we want their total number of transactions.  {'Sam': 5, 'Alex': 2424, ...}

In [1]:
from utils import increment_dictionary, sort_dictionary_by_values, path_csv, get_data
from csv_utils import make_csv
import pandas as pd 

def num_of_trans_per_person():
    title, rows = get_data()
    d={}
    
    for _, transaction in rows:
        d = increment_dictionary(d, transaction[title])

    filename = "num_of_trans_per_person"
    key_header = title
    value_header = "number_of_transactions"

    d = sort_dictionary_by_values(d)
    
    wd = make_csv(path_csv, filename, d, [key_header, value_header])
    df = pd.read_csv(wd)
    print(df.head(5))

    return d 

trans_per_person_res = num_of_trans_per_person()


               senator  number_of_transactions
0  David A Perdue , Jr                    2597
1      Thomas R Carper                     970
2   Sheldon Whitehouse                     693
3          Pat Roberts                     477
4      Susan M Collins                     449


# Number of Transactions per Person Controlled
_Divide number of transactions by number of years in official position.  Not controlling for size of transaction._

In [2]:
from wiki import wiki_search
from utils import get_data
import math 

def num_of_trans_per_person_controlled(rows):    
    title, _ = get_data()
    d={}

    for person, num_of_trans in rows.items():
        official = wiki_search(person)       
        years_in_office = official.get_num_of_years()
        d[person] = math.ceil(num_of_trans/years_in_office) 

    filename = "num_of_trans_per_person_controlled"
    value_header = "avg_number_of_transactions"

    d = sort_dictionary_by_values(d)
   
        
    wd = make_csv(path_csv, filename, d, [title, value_header])
    print(pd.read_csv(wd).head(5))
    
    return d 

num_of_trans_per_person_controlled_res = num_of_trans_per_person_controlled(trans_per_person_res)
    

               senator  avg_number_of_transactions
0  David A Perdue , Jr                         433
1       Kelly Loeffler                         279
2  Thomas H Tuberville                         219
3           Rick Scott                          76
4     Shelley M Capito                          63


# Number of Transactions per Person Controlled w/Affiliation 

In [3]:
from wiki import wiki_search
import math 

def num_of_trans_per_person_controlled_w_aff(rows):
    title, _ = get_data()
    d={}
    
    for person, val in rows.items():
        official = wiki_search(person)     
        d[person] = val, official.get_party()

    filename = "num_of_trans_per_person_controlled_w_aff"
    value_header = "avg_number_of_transactions"

    d = sort_dictionary_by_values(d)
        
    wd = make_csv(path_csv, filename, d, [title, value_header, "party"])
    print(pd.read_csv(wd).head(5))
    
    return d, pd.read_csv(wd).head(10)

num_of_trans_per_person_controlled_w_aff_res, top_10 = num_of_trans_per_person_controlled_w_aff(num_of_trans_per_person_controlled_res)
    

               senator  avg_number_of_transactions  \
0  David A Perdue , Jr                         433   
1       Kelly Loeffler                         279   
2  Thomas H Tuberville                         219   
3           Rick Scott                          76   
4     Shelley M Capito                          63   

                              party  
0  Republican Party (United States)  
1  Republican Party (United States)  
2  Republican Party (United States)  
3  Republican Party (United States)  
4  Republican Party (United States)  


# Number of Transactions per Person by Date 

In [4]:
from utils import increment_dictionary_in_dictionary, path_csv
from csv_utils import make_csv_breakdown

def num_of_trans_per_person_per_date():    
    title, rows = get_data()
    d={}

    for _, transaction in rows:
        date = transaction['transaction_date']
        d = increment_dictionary_in_dictionary(d, transaction[title], date)

    filename = "num_of_trans_per_person_per_date"
    
    
    # some sort of error 
    wd = make_csv_breakdown(path_csv, filename, d, title)
    print(pd.read_csv(wd).head(5))

num_of_trans_per_person_per_date()


               senator  01/01/2014  01/02/2013  01/02/2014  01/02/2015  \
0  Thomas H Tuberville         NaN         NaN         NaN         NaN   
1     Cynthia M Lummis         NaN         NaN         NaN         NaN   
2      Thomas R Carper         NaN         NaN         1.0         NaN   
3            Roy Blunt         NaN         NaN         NaN         NaN   
4     Shelley M Capito         NaN         NaN         NaN         NaN   

   01/02/2018  01/02/2020  01/03/2014  01/03/2017  01/03/2018  ...  \
0         NaN         NaN         NaN         NaN         NaN  ...   
1         NaN         NaN         NaN         NaN         NaN  ...   
2         NaN         2.0         1.0         NaN         NaN  ...   
3         NaN         NaN         NaN         NaN         NaN  ...   
4         NaN         NaN         NaN         NaN         NaN  ...   

   12/30/2014  12/30/2015  12/30/2016  12/30/2019  12/30/2020  12/31/2014  \
0         NaN         NaN         NaN         NaN        

# Number of Transactions per Date 

In [5]:
from graph_utils import graph_csv 
from csv_utils import make_csv
from utils import path_html, increment_dictionary, path_csv, reversedict
import plotly.express as px

def num_of_trans_per_person_per_date():
    _, rows = get_data()
    d={}

    for _, transaction in rows:
        date = transaction['transaction_date']
        d = increment_dictionary(d, date)

    filename = "num_of_trans_per_person_per_date"
    key_header = "date"
    value_header = "number_of_transactions"
    
    d = reversedict(d)
    
    make_csv(path_csv, filename, d, [key_header, value_header])
    fig = graph_csv(path_csv, path_html, filename, key_header, value_header)
    fig.show()
    return d 

trans_per_person_per_date_res = num_of_trans_per_person_per_date()

# Frequency of Transactions per Date Controlled
_Number of transactions per date controlled by official. E.g. if Ted Baker made 40 transactions on 1/1/02 and Sam Wall made 2 transactions on 1/1/02, we conclude that there were two transactions on 1/1/02._


In [6]:
from utils import increment_dictionary_in_dictionary, path_csv, path_html, reversedict, get_data
from csv_utils import make_csv
from graph_utils import graph_csv

def num_of_trans_per_date_controlled():
    title, rows = get_data()
    d = {}

    for _, transaction in rows:
        date = transaction['transaction_date']
        d = increment_dictionary_in_dictionary(d, date, transaction[title])
    
    d2 = {}
    for date in d:
        d2[date] =  len(d[date])
        
    d2 = reversedict(d2)
        
    filename = "num_of_trans_per_date_controlled"
    key_header = "date"
    value_header = "number_of_transactions_unique"

    make_csv(path_csv, filename, d2, [key_header, value_header])
    fig = graph_csv(path_csv, path_html, filename, key_header, value_header, scatter=True)
    fig.show()
    return d 
    
num_of_trans_per_date_controlled_res = num_of_trans_per_date_controlled()


# Number of Transactions Within 2 Weeks Prior to Quarterly Tax Date 

In [7]:
from utils import within_tax_date, increment_dictionary
from wiki import wiki_search

# i should like type of transactions @TODO. 

def num_of_trans_within_tax_date(rows):
        total = within = 0 
        
        for k,v in rows.items():  
                total += 1 
                if within_tax_date(k):
                        within += v 

        print("Percent of transactions posted within two weeks of quarterly tax deadline: {percent}%".format(percent=str((within/total)*100)[:5]))
        return (within/total)*100

frac = num_of_trans_within_tax_date(trans_per_person_per_date_res)


Percent of transactions posted within two weeks of quarterly tax deadline: 81.11%


# Number of Transactions Within 2 Weeks Prior to Quarterly Tax Date Semi-Controlled 

_Given dict='09/03/2021': {'Thomas H Tuberville': 1, 'Cynthia M Lummis': 1, 'A. Mitchell Mcconnell, Jr.': 1}...I only incremement the number of within (tax date) once per date per official. So, if an official does 100 transactions on a date within two weeks of a quarterly deadline, then I only count it as one transaction._

_A Note: total === number of transactions per person per date (so not really all transactions) because someone could have potentially made 60 transactions on one date which we don't include in neither total or within, if applicable._

In [8]:
def num_of_trans_within_tax_date_controlled(rows):
        total = within = 0 
        people = set()

        for date, inner_dict in rows.items():
                if within_tax_date(date):
                    for person in inner_dict:
                                people.add(person)
                                within += 1 
                                total += 1 
                else:
                        for person in inner_dict:
                                total += 1 
     
     
        print("Percent of transactions posted within two weeks of quarterly tax deadline: {percent}%".format(percent=str((within/total)*100)[:5]))
        return people

num_of_trans_within_tax_date_controlled_res = num_of_trans_within_tax_date_controlled(num_of_trans_per_date_controlled_res)


Percent of transactions posted within two weeks of quarterly tax deadline: 15.18%


In [9]:
def people_and_within_tax_date(people):        
        # todo get number of senators. 
        # todo is the monetary value of that equal!!!! 
        d = {}
        for i in people:
                d[i] = ""
                
        wd = make_csv(path_csv, "people_and_within_tax_date_list", d, ["Officials"])
        df = pd.read_csv(wd)
        print("People who posted transactions within two weeks of quarterly tax deadline:\n {}\n".format(df.head(5)))

        print("Number of people who posted transactions within two weeks of quarterly tax deadline: {}\n".format(len(people)))
        
        party = {}
        for p in people:
                party = increment_dictionary(party, wiki_search(p).get_party())
                
        party = sort_dictionary_by_values(party)
        
        wd = make_csv(path_csv, "people_and_within_tax_date_list_w_aff", party, ["party", "number_of_filing_within_tax_date"])
        df = pd.read_csv(wd)
        print("Party breakdown of people who posted transactions within two weeks of quarterly tax deadline:\n {}\n".format(df.head(5)))

people_and_within_tax_date(num_of_trans_within_tax_date_controlled_res)

People who posted transactions within two weeks of quarterly tax deadline:
                      Officials
William Cassidy            NaN
Joseph Manchin, Iii        NaN
Steve Daines               NaN
Maria Cantwell             NaN
Michael  B Enzi            NaN

Number of people who posted transactions within two weeks of quarterly tax deadline: 47

Party breakdown of people who posted transactions within two weeks of quarterly tax deadline:
                               party  number_of_filing_within_tax_date
0  Republican Party (United States)                                24
1  Democratic Party (United States)                                22
2            Independent politician                                 1



In [10]:
def people_and_within_tax_date_how_often(people):
        title, rows = get_data()
        d = {}
        d_controlled_by_dates = {}
        for _, transaction in rows:
                if transaction[title] in people and within_tax_date(transaction['transaction_date']):
                        d = increment_dictionary(d, transaction[title])
                        d_controlled_by_dates = increment_dictionary_in_dictionary(d_controlled_by_dates, transaction['transaction_date'], transaction[title])

        d_controlled_by_dates_res  = {}
        for date in d_controlled_by_dates:
                for person in d_controlled_by_dates[date]:
                        d_controlled_by_dates_res = increment_dictionary(d_controlled_by_dates_res, person)

        d = sort_dictionary_by_values(d)
        d_controlled_by_dates_res = sort_dictionary_by_values(d_controlled_by_dates_res)

        wd = make_csv(path_csv, "people_and_within_tax_date_how_often", d, [title, "number_of_filing_within_tax_date"])
        df = pd.read_csv(wd)
        print("People who posted transactions within two weeks of quarterly tax deadline and the number of transactions posted:\n {}\n".format(df.head(5)))

        wd = make_csv(path_csv, "people_and_within_tax_date_how_often_date_controlled", d_controlled_by_dates_res, [title, "number_of_filing_within_tax_date_date_controlled"])
        df = pd.read_csv(wd)
        print("People who posted transactions within two weeks of quarterly tax deadline and the number of transactions posted controlled by date:\n {}\n".format(df.head(5)))

          
people_and_within_tax_date_how_often(num_of_trans_within_tax_date_controlled_res)



People who posted transactions within two weeks of quarterly tax deadline and the number of transactions posted:
                senator  number_of_filing_within_tax_date
0  David A Perdue , Jr                               368
1      Thomas R Carper                               179
2       Kelly Loeffler                               155
3     Shelley M Capito                               139
4   Sheldon Whitehouse                               102

People who posted transactions within two weeks of quarterly tax deadline and the number of transactions posted controlled by date:
                senator  number_of_filing_within_tax_date_date_controlled
0  David A Perdue , Jr                                               113
1      Thomas R Carper                                                52
2          Pat Roberts                                                44
3   Sheldon Whitehouse                                                28
4      Susan M Collins                       

# Frequency of Differences between Transaction and Disclosure Date

In [8]:
from utils import difference_between_dates, increment_dictionary, sort_dictionary_by_keys, path_csv, path_html
from graph_utils import graph_csv
from csv_utils import make_csv

# todo, check if outlier exists (manually check it).
# potential for automating it? ---> policy issue?

def frequency_of_differences(d={}):
    _, rows = get_data()
    total = num = 0 

    for _, transaction in rows:
        transaction_date = transaction['transaction_date']
        disclosure_date = transaction['disclosure_date']

        # Negative, X days BEFORE
        # Positive, Y dayas AFTER
        diff = difference_between_dates(disclosure_date, transaction_date)      
        total += 1 
        num += diff   
        d = increment_dictionary(d, int(diff))
    
    d = sort_dictionary_by_keys(d)
        
    filename = "frequency_of_differences"
    key_header = "difference_in_days"
    value_header = "#_of_transactions_with_that_diff"
    make_csv(path_csv, filename, d, [key_header, value_header])
    fig = graph_csv(path_csv, path_html, filename, key_header, value_header)
    fig.show()
    
    print("Average difference in days: {}".format(num//total))

frequency_of_differences()

Average difference of days: 30


# Frequency of Actions

In [12]:
from utils import get_data, isvalid

def frequency_of_act(d={}):
    _, rows = get_data()

    for _, transaction in rows:
        if isvalid(transaction['type']): 
            d = increment_dictionary(d, transaction['type'])
    
    d = sort_dictionary_by_values(d)
    
    filename = "frequency_of_act"
    key_header = "type"
    value_header = "number_of_transactions"
    wd = make_csv(path_csv, filename, d, [key_header, value_header])
    df = pd.read_csv(wd)
    print(df.head(5))

     
frequency_of_act()

             type  number_of_transactions
0        Purchase                    4350
1     Sale (Full)                    2212
2  Sale (Partial)                    1756
3        Exchange                      96


# Frequency of Asset Type

In [13]:
def frequency_of_asset_type(d={}):
    _, rows = get_data()

    for _, transaction in rows:
        if isvalid(transaction['asset_type']):
            d = increment_dictionary(d, transaction['asset_type'])
      
    d = sort_dictionary_by_values(d)
  
    filename = "frequency_of_asset_type"
    key_header = "asset_type"
    value_header = "number_of_transactions"
    wd = make_csv(path_csv, filename, d, [key_header, value_header])
    df = pd.read_csv(wd)
    print(df.head(5))


frequency_of_asset_type()

             asset_type  number_of_transactions
0                 Stock                    6486
1  PDF Disclosed Filing                     465
2    Municipal Security                     380
3      Other Securities                     359
4        Corporate Bond                     232


# Frequency of Transactions by Political Affiliation

In [14]:
from wiki import wiki_search
from utils import increment_dictionary_in_dictionary, add_sort_key_for_amount, get_data, sort_dictionary_by_sort_key, path_csv 
from csv_utils import make_csv_breakdown
import pandas as pd 

def frequency_of_amount_by_aff(d={}):
    _, rows = get_data()

    affiliations = {}
    for _, transaction in rows:
        person = transaction['senator']
        
        if person not in affiliations: 
            rep = wiki_search(person) 
            affiliations.update({person : rep.party})
            
        d = increment_dictionary_in_dictionary(d, transaction['amount'], affiliations[person])

    
    d = add_sort_key_for_amount(d)
    d = sort_dictionary_by_sort_key(d)
    
        
    filename = "frequency_of_amount_by_aff"
    key_header = "amount"
    
    wd = make_csv_breakdown(path_csv, filename, d, key_header)
    print(pd.read_csv(wd).head(2))
   
    
frequency_of_amount_by_aff()

             amount  Democratic Party (United States)  Independent politician  \
0           Unknown                             189.0                     8.0   
1  $1,001 - $15,000                            1981.0                    51.0   

   Republican Party (United States)  sort_key  
0                               268        -1  
1                              3714     15000  


# Frequency of Tickers 
_Only tracking valid ones._

In [15]:
from utils import isvalid 

def frequency_of_ticker(d={}):
    _, rows = get_data()

    # Only tracking the valid ones. 
    for _, transaction in rows:
        if isvalid(transaction['ticker']):
            d = increment_dictionary(d, transaction['ticker'])

    d = sort_dictionary_by_values(d)
    
    filename = "ticker_total"
    key_header = "ticker"
    value_header = "number_of_transactions"
    make_csv(path_csv, filename, d, [key_header, value_header])
    fig = graph_csv(path_csv, path_html, filename, key_header, value_header) 
    fig.show()

frequency_of_ticker()

# Frequency of Ticker per Year

In [6]:
from utils import isvalid, get_data, sort_dictionary_by_keys, get_year, increment_dictionary_in_dictionary, path_csv
from csv_utils import make_csv_breakdown
import pandas as pd 

# group by industry then do frequency. 
def frequency_of_ticker_breakdown_ticker(d={}):
    _, rows = get_data()

    for _, transaction in rows:
        if isvalid(transaction['ticker']):
            d = increment_dictionary_in_dictionary(d, transaction['ticker'], get_year(transaction['transaction_date']))

    d = sort_dictionary_by_keys(d)
    
    filename = "trans_per_year_breakdown"
    key_header = "ticker"
    
    wd = make_csv_breakdown(path_csv, filename, d, key_header)
    print(pd.read_csv(wd).head(2))

frequency_of_ticker_breakdown_ticker()

    ticker  2012  2013  2014  2015  2016  2017  2018  2019  2020  2021
0  0QZI.IL   NaN   NaN   NaN   NaN   NaN   NaN   NaN   2.0   NaN   NaN
1  3V64.TI   NaN   NaN   NaN   NaN   NaN   NaN   NaN   2.0   NaN   NaN


# Frequency of Ticker per Date

In [17]:
from utils import get_year, increment_dictionary_in_dictionary, path_csv
from csv_utils import make_csv_breakdown

def frequency_of_ticker_by_date(d={}):
    _, rows = get_data()
    
    for _, transaction in rows:
        d = increment_dictionary_in_dictionary(d, transaction['ticker'], transaction['transaction_date'])

    filename = "frequency_of_ticker_by_date"
    key_header = "ticker"
    
    wd = make_csv_breakdown(path_csv, filename, d, key_header)
    print(pd.read_csv(wd).head(2))

frequency_of_ticker_by_date()

  ticker  01/01/2014  01/02/2013  01/02/2014  01/02/2015  01/02/2018  \
0     PG         NaN         NaN         NaN         NaN         NaN   
1   NFLX         NaN         NaN         NaN         NaN         NaN   

   01/02/2020  01/03/2014  01/03/2017  01/03/2018  ...  12/30/2014  \
0         NaN         NaN         NaN         NaN  ...         NaN   
1         NaN         NaN         NaN         NaN  ...         NaN   

   12/30/2015  12/30/2016  12/30/2019  12/30/2020  12/31/2014  12/31/2015  \
0         NaN         NaN         NaN         NaN         NaN         NaN   
1         NaN         NaN         NaN         NaN         NaN         NaN   

   12/31/2018  12/31/2019  12/31/2020  
0         NaN         NaN         NaN  
1         NaN         NaN         NaN  

[2 rows x 1833 columns]


# Number of Transactions per Industry
_Not controlled in any way._

In [18]:
from utils import increment_dictionary, sort_dictionary_by_values, path_csv, get_data, isvalid, get_mapping, search_mapping
from csv_utils import make_csv
import pandas as pd 

def number_of_transactions_per_indusry():        
    d = {}
    _, rows = get_data()
    df = get_mapping()
    
    for _, transaction in rows:
        ticker = transaction['ticker']
        industry = search_mapping(df, ticker)
        if industry: 
            d = increment_dictionary(d, industry)

    filename = "number_of_transactions_per_indusry"
    key_header = "industry"
    value_header = "number_of_transactions"

    d = sort_dictionary_by_values(d)
    wd = make_csv(path_csv, filename, d, [key_header, value_header])
    df = pd.read_csv(wd)
    print(df.head(5))    
    
    return d 

number_of_transactions_per_indusry_res = number_of_transactions_per_indusry()

                     industry  number_of_transactions
0                        Fund                     517
1               Entertainment                     355
2  Drug Manufacturers—General                     300
3         Oil & Gas Midstream                     219
4     Software—Infrastructure                     213


# Frequency of Industry per Year

In [21]:
from utils import get_year, increment_dictionary_in_dictionary, path_csv
from csv_utils import make_csv_breakdown

def frequency_of_industry_breakdown_ticker(d={}):
    _, rows = get_data()
    df = get_mapping()

    for _, transaction in rows:
        
        industry = search_mapping(df, transaction['ticker'])
        if industry: 
            d = increment_dictionary_in_dictionary(d, industry, get_year(transaction['transaction_date']))

    filename = "frequency_of_industry_breakdown_year"
    key_header = "industry"
    
    wd = make_csv_breakdown(path_csv, filename, d, key_header)
    print(pd.read_csv(wd).head(2))

frequency_of_industry_breakdown_ticker()

                        industry  2012  2013  2014  2015  2016  2017  2018  \
0  Household & Personal Products   NaN   NaN  14.0  12.0  17.0  19.0  15.0   
1                  Entertainment   NaN   NaN  14.0  30.0  38.0  44.0  95.0   

   2019  2020  2021  
0   9.0  18.0   6.0  
1  47.0  82.0   5.0  


# Industry Breakdown per Official

In [23]:
from utils import get_year, increment_dictionary_in_dictionary, path_csv
from csv_utils import make_csv_breakdown

def frequency_of_industry_breakdown_official(d={}):
    title, rows = get_data()
    df = get_mapping()

    for _, transaction in rows:
        
        industry = search_mapping(df, transaction['ticker'])
        if industry: 
            d = increment_dictionary_in_dictionary(d, transaction[title], industry)

    filename = "frequency_of_industry_per_official"

    wd = make_csv_breakdown(path_csv, filename, d, title)
    print(pd.read_csv(wd).head(2))

frequency_of_industry_breakdown_official()

               senator  Aerospace & Defense  Agricultural Inputs  Airlines  \
0  Thomas H Tuberville                  NaN                  NaN       NaN   
1     Cynthia M Lummis                  NaN                  NaN       NaN   

   Aluminum  Apparel Manufacturing  Apparel Retail  Asset Management  \
0      12.0                    NaN             1.0               1.0   
1       NaN                    NaN             NaN               NaN   

   Auto & Truck Dealerships  Auto Manufacturers  ...  Travel Services  \
0                       NaN                 2.0  ...              NaN   
1                       NaN                 NaN  ...              NaN   

   Trucking  Type  Utilities—Diversified  Utilities—Regulated Electric  \
0       1.0   NaN                    NaN                           NaN   
1       NaN   NaN                    NaN                           NaN   

   Utilities—Regulated Gas  Utilities—Regulated Water  Utilities—Renewable  \
0                      NaN 